# **Tratamiento bases de datos**

## **Preguntas claves**

**¿Existen patrones espaciales y temporales que se repiten en el tiempo?**

**¿Existe relación entre la estratificación del área donde se registra el hurto con la cantidad de hurtos registrados?**

**¿Hay baja frecuencia de hurtos en las zonas monitorizadas con cámaras CCTV de la secretaría de movilidad de Medellín?**

**¿Existe relación entre el área de recuperación de una motocicleta con la estratificación de esta área?**



---
## ***Cargue de librerias***

In [ ]:
!pip install unidecode

In [ ]:
import json
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats import zscore, median_abs_deviation
import unidecode
import plotly.express as px
import plotly.graph_objects as go

---
## ***Creación de un diccionario con las url de las bases de datos***

In [ ]:
url = {}
url['Camaras'] = 'https://www.datos.gov.co/resource/6ic8-r398.json'
url['Hurto_motos'] = 'https://medata.gov.co/sites/default/files/distribution/1-027-23-000014/hurto_de_moto.csv'
url['Recuperacion'] = 'https://medata.gov.co/sites/default/files/distribution/1-027-23-000309/recuperacion_de_automotores_anio.csv'
url['Estratos'] = 'https://www.medellin.gov.co/es/wp-content/uploads/2022/07/EstadisticasWEB2023-Diciembre.xlsx'

## ***Lectura de las url y creación de los DataFrame***

### Lectura de urls

In [ ]:
dfs = {}

for name, url in url.items():
    if url.endswith('.json'):
        dfs[name] = pd.read_json(url)
    elif url.endswith('.csv'):
        dfs[name] = pd.read_csv(url)
    elif url.endswith('.xlsx'):
        sheet_name1 = 'Estrato Predominante Barrio'
        sheet_name2 = 'Estrato Predominante Comuna'
        dfs[sheet_name1] = pd.read_excel(url, sheet_name = 'Estrato Predominante Barrio')
        dfs[sheet_name2] = pd.read_excel(url, sheet_name = 'Estrato Predominante Comuna')

En el caso de la base de datos 'Estratos por barrio', reuqerimos de hacer unas pequeñas modificaciones antes de crear el dataframe

In [ ]:
# Remover la primera fila que se encuentra totalmente vacia
dfs['Estrato Predominante Barrio'] = dfs['Estrato Predominante Barrio'].iloc[0:]
# Configurar la "nueva" primer fila como los encabezados de cada columna
dfs['Estrato Predominante Barrio'].columns = dfs['Estrato Predominante Barrio'].iloc[0]
# Remover la segunda fila
dfs['Estrato Predominante Barrio'] = dfs['Estrato Predominante Barrio'].iloc[1:]

### Creación de los Data Frame

In [ ]:
df_cam = pd.DataFrame(dfs['Camaras'])
df_hurto = pd.DataFrame(dfs['Hurto_motos'])
df_rec = pd.DataFrame(dfs['Recuperacion'])
df_estratos_barrio = pd.DataFrame(dfs['Estrato Predominante Barrio'])
df_estratos_comuna = pd.DataFrame(dfs['Estrato Predominante Comuna'])

---
## ***Identidicación de variables no relevantes, outliers, duplicados y valores nulos***

---
### **Base de datos de las camaras de la ciudad**

In [ ]:
df_cam.shape

In [ ]:
df_cam.info()

In [ ]:
df_cam

#### **Información no relevante**

Al revisar el DataFrame, notamos que las columnas:


1. 'link_detalle'
2. 'link_foto'
3. 'point'
4. 'latitud'
5. 'longitud'

No aportan información relevante para el modelo por lo cual tomamos la decisión de eliminarlas.

1. 'objectid'
2. 'x_magnamed'
3. 'y_magnamed'

Son columnas de las cuáles no tenemos información y desconocemos su significado, por ende no utilizaremos.


Eliminamos las columnas mencionadas anteriormente y obtenemos el dataframe.

In [ ]:
#df_cam1, copia del dataframe original, para manipularla
df_cam1 = df_cam.drop(columns = ['link_detalle', 'link_foto', 'point', 'x_magnamed', 'y_magnamed', 'objectid'])
df_cam1

#### **Agregar las columnas comuna y barrio**

Hemos decidio realizar el cruce de bases guiándonos de las variables 'barrio' o 'comuna', por ende agregamos estas mismas a las cámaras buscando sus ubicaciones de manera manual.

In [ ]:
barrios = ('LA CANDELARIA', 'VILLA CARLOTA','EL POBLADO','SANTA MARIA DE LOS ANGELES','SANTA MARIA DE LOS ANGELES','VILLA NUEVA','LAURELES',
'CASTROPOL','ALTOS DEL POBLADO','SAN DIEGO','UNIVERSIDAD NACIONAL','LA PILARICA','CASTILLA','BELALCAZAR','CRISTO REY','CORAZON DE JESUS',
'CORAZON DE JESUS','LA AGUACATALA','GUAYABAL','SANTA FE','EL ESTADIO','LA AGUACATALA','GUAYABAL','GUAYABAL','EL TESORO','EL POBLADO','CAMPO AMOR',
'LAS LOMAS # 1','EL POBLADO','SANTA FE','CASTROPOL','AEROPARQUE JUAN PABLO II','VILLA CARLOTA','SAN DIEGO','EL SOCORRO','LA PALMA','BARRIO COLON',
'LOS CONQUISTADORES','BOMBONA # 1','LOS CONQUISTADORES','EL ESTADIO','LA CANDELARIA','SURAMERICANA','EL CHAGUALO','CARLOS E. RESTREPO','MORAVIA',
'OLEODUCTO','TRICENTENARIO','CASTILLA','BOYACA','LAS BRISAS','EL DIAMANTE','LA FLORIDA','BELALCAZAR','LA AGUACATALA','CARIBE','SANTA FE','OLEODUCTO',
'PATIO BONITO','ENVIGADO','PROGRESO','SAN BENITO','LA AGUACATALA','SANTA FE','EL CHAGUALO','PERPETUO SOCORRO','CORAZON DE JESUS','HECTOR ABAD GOMEZ',
'SAN CRISTOBAL','PROGRESO','SANTA MARGARITA','EL DANUBIO','NUEVA VILLA DE ABURRA')

comunas = ('LA CANDELARIA', 'EL POBLADO', 'EL POBLADO', 'EL POBLADO', 'EL POBLADO','LA CANDELARIA', 'LAURELES', 'EL POBLADO', 'EL POBLADO',
'LA CANDELARIA', 'ROBLEDO', 'ROBLEDO','CASTILLA', 'CASTILLA', 'GUAYABAL', 'LA CANDELARIA', 'LA CANDELARIA', 'EL POBLADO', 'GUAYABAL','GUAYABAL',
'LAURELES', 'EL POBLADO', 'GUAYABAL', 'GUAYABAL', 'EL POBLADO', 'EL POBLADO', 'GUAYABAL','EL POBLADO', 'EL POBLADO', 'GUAYABAL', 'EL POBLADO',
'GUAYABAL', 'EL POBLADO', 'LA CANDELARIA', 'SAN JAVIER','BELEN', 'LA CANDELARIA', 'LAURELES', 'LA CANDELARIA', 'LAURELES', 'LAURELES',
'LA CANDELARIA', 'LAURELES','LA CANDELARIA', 'LAURELES', 'ARANJUEZ', 'CASTILLA', 'CASTILLA', 'CASTILLA', 'CASTILLA', 'CASTILLA', 'ROBLEDO',
'EL POBLADO', 'CASTILLA', 'EL POBLADO', 'CASTILLA', 'GUAYABAL', 'CASTILLA', 'EL POBLADO', 'ENVIGADO', 'CASTILLA','LA CANDELARIA', 'EL POBLADO',
'GUAYABAL', 'LA CANDELARIA', 'LA CANDELARIA', 'LA CANDELARIA', 'CASTILLA','SAN CRISTOBAL', 'CASTILLA', 'ROBLEDO', 'LA AMERICA', 'BELEN')

In [ ]:
df_cam1['comuna'] = comunas
df_cam1['barrio'] = barrios
df_cam1

Luego de ubicar las cámaras por barrio y comuna, nos damos cuenta que las cámaras 60   (ENVIGADO) y 69 (SAN CRISTOBAL) se encuentran fuera de Medellín que es el área delimitada, por ende, las suprimimos del dataframe.

In [ ]:
df_cam1 = df_cam1.drop([59, 68])

#### **Valores duplicados**


In [ ]:
df_cam1.duplicated().sum()

#### **Valores nulos**

In [ ]:
df_cam1.isnull().sum()

La base de datos de las cámaras de seguridad no cuenta con valores nulos y tampoco con valores duplicados.

---
### **Base de datos de hurtos de motos en la ciudad**

In [ ]:
# Copia del DataFrame original para su manipulación
df_hurto1 = df_hurto

In [ ]:
df_hurto1.shape

#### **Información no relevante**

Hemos decidido trabajar con un nivel de detalle de barrios, por lo que no se tendra en cuenta el dato de las coordenadas (latitud y longitud) del hecho y se toma la decición de eliminar estas columnas.

**Las columnas:**

1. 'grupo_actor'
2. 'actividad_delictiva'
3. 'parentesco'
4. 'unidad_medida'
5. 'discapacidad'
6. 'ocupacion'
7. 'grupo_especial'
8. 'nivel_academico'
9. 'testigo'
10. 'caracterizacion'
11. 'articulo_penal'
12. 'categoria_penal'
13. 'permiso'


Contienen un único valor 'Sin dato', por lo que se decide eliminarlas.

**Las columnas:**

1. 'estado_civil'
2. 'medio_transporte'
3. 'conducta'
4. 'sede_receptora'
5. 'grupo_bien'

Contienen valores que consideramos no son necesarios para el análisis, por lo que también son eliminadas.

Y finalmente, al revisar la información de la columna 'fecha_ingestion' notamos que esta solo tiene una única fecha de abril de 2024, por lo que no la consideramos relevente y decidimos eliminarla, al igual que la columna 'cantidad' que hace referencia a la cantidad de vehículos hurtados por cada registro, pero esta tiene un único valor (1) para las motocicletas, así que también decidimos eliminarla.

In [ ]:
df_hurto1 = df_hurto1.drop(columns = ['grupo_actor', 'actividad_delictiva', 'parentesco', 'unidad_medida',
                                      'discapacidad', 'ocupacion', 'grupo_especial', 'nivel_academico', 'testigo',
                                      'caracterizacion', 'articulo_penal', 'categoria_penal', 'permiso','estado_civil', 'medio_transporte',
                                      'conducta', 'sede_receptora', 'grupo_bien','fecha_ingestion', 'cantidad'])

In [ ]:
df_hurto1['bien'].unique()

Ahora, notamos que existen registros de hurtos para bienes que no estamos tratando en este trabajo como lo son: Carne, Bicicleta, Sin dato, Motor vehículos y Moto carro, así que decidimos suprimir las filas con los índices correspondientes a estos datos.

In [ ]:
#se halla una carne hurtada
carne = df_hurto1[df_hurto1['bien'] == 'Carne']
indicecarne = list(carne.index)
print(len(indicecarne))

In [ ]:
#se encuentran 5809 registros donde no se especifica el bien hurtado
sindato = df_hurto1[df_hurto1['bien'] == 'Sin dato']
indicesindato = list(sindato.index)
print(len(indicesindato))

In [ ]:
#se halla una bicileta hurtada
bicicleta = df_hurto1[df_hurto1['bien'] == 'Bicicleta']
indicebicicleta = list(bicicleta.index)
print(len(indicebicicleta))

In [ ]:
#se halla un motorvehiculo hurtado
motorvehiculo = df_hurto1[df_hurto1['bien'] == 'Motor vehículo']
indicemotorvehiculo = list(motorvehiculo.index)
print(len(indicemotorvehiculo))

In [ ]:
##se encuentran 69 moto carros hurtados
motocarro = df_hurto1[df_hurto1['bien'] == 'Moto carro']
indicemotocarro = list(motocarro.index)
print(len(indicemotocarro))

In [ ]:
#se eliminan 5881 fias de registros de hurtos que no son de interes para el trabajo
indices_suprimir = indicecarne + indicebicicleta + indicesindato + indicemotorvehiculo + indicemotocarro
print(len(indices_suprimir))

In [ ]:
df_hurto1 = df_hurto1.drop(indices_suprimir)

In [ ]:
#finalmente la base de datos pasó de 86.410 a 80.529 filas y de 36 a 14 columnas con las que vamos a trabajar,
#eliminando 22 columnas y 5881 registros irrelevantes
df_hurto1.shape

#### **Redefinición de variables**

Para un correcto cruce con los demas dataframe, Hemos puesto el texto en mayúsculas y sin tildes para la columna 'nombre_barrio'.

In [ ]:
df_hurto1['nombre_barrio'] = df_hurto1['nombre_barrio'].str.upper()
df_hurto1['nombre_barrio'] = df_hurto1['nombre_barrio'].apply(unidecode.unidecode)
df_hurto1['nombre_barrio']

Reemplazamos la columna 'codigo_comuna' por 'nombre_comuma', ademas de ponerla en el formato necesario para cruzarla con los demas DataFrame.

In [ ]:
df_hurto1.insert(9, 'codigo_comuna1', df_hurto1['codigo_comuna'].copy())

In [ ]:
df_hurto1['codigo_comuna1'].unique()

In [ ]:
df_hurto1['codigo_comuna1'] = df_hurto1['codigo_comuna1'].replace({
                                              1:'1', 2:'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8',
                                              9:'9', 10:'10', 11:'11', 12:'12', 13:'13', 14:'14', 15:'15',
                                              16:'16', 50:'50', 60:'60', 70:'70', 80:'80', 90:'90'
                                              })

In [ ]:
df_hurto1['codigo_comuna'] = df_hurto1['codigo_comuna'].replace({
                                              1:'1', 2:'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8',
                                              9:'9', 10:'10', 11:'11', 12:'12', 13:'13', 14:'14', 15:'15',
                                              16:'16', 50:'50', 60:'60', 70:'70', 80:'80', 90:'90'
                                              })

In [ ]:
df_hurto1['codigo_comuna1'] = df_hurto1['codigo_comuna1'].replace({
                                              '1':'POPULAR', '2':'SANTA CRUZ', '3':'MANRIQUE', '4':'ARANJUEZ',
                                              '5':'CASTILLA', '6':'DOCE DE OCTUBRE', '7':'ROBLEDO', '8':'VILLA HERMOSA',
                                              '9':'BUENOS AIRES', '10':'LA CANDELARIA', '11':'LAURELES', '12':'LA AMERICA',
                                              '13':'SAN JAVIER', '14':'EL POBLADO', '15':'GUAYABAL', '16':'BELEN',
                                              '50':'PALMITAS', '60':'SAN CRISTOBAL', '70':'ALTAVISTA', '80':'SAN ANTONIO DE PRADO',
                                              '90':'SANTA ELENA'
                                              })

Se elimanan las filas del df en donde el valor de la columa 'codigo_comuna' es 'Sin dato'.

In [ ]:
df_hurto1 = df_hurto1[df_hurto1['codigo_comuna1'] != 'SIN DATO']

In [ ]:
df_hurto1['codigo_comuna1'].unique()

In [ ]:
df_hurto1 = df_hurto1.rename(columns = {'codigo_comuna1' : 'nombre_comuna'})

##### **Eliminación de los corregimientos de la base de datos**

Nuestra área de influencia es la ciudad de Medellín sin tener en cuenta los corregimientos que hacen parte de ella, es por ello que se eliminan los 5 corregimientos de los registros

In [ ]:
df_hurto1['nombre_comuna'].unique()

In [ ]:
corre = ['ALTAVISTA', 'SAN CRISTOBAL', 'SAN ANTONIO DE PRADO', 'SANTA ELENA', 'PALMITAS']
for i in corre:
  df_hurto1.drop(df_hurto1[df_hurto1['nombre_comuna'] == i].index, inplace=True)

In [ ]:
df_hurto1['nombre_comuna'].unique()

#### **Valores duplicados**

In [ ]:
a = len(df_hurto1)
a

In [ ]:
b = df_hurto1.duplicated().sum()
b

In [ ]:
b/a

Los datos duplicados representan el 9,93% del total de datos, consideramos que se puede eliminar.

In [ ]:
df_hurto1 = df_hurto1.drop_duplicates()

#### **Valores nulos**

In [ ]:
df_hurto1.isnull().sum()

La base de datos no cuento con dato nulos.

#### **Categorización**

Separación de la variable 'fecha_hecho' en sus componentes: año, mes, día y hora. Además de la creación de una nueva variable que según el día del hecho diferencie si el hurto ocurrió un fin de semana (entre viernes y domingo) o no (entre lunes y jueves).

In [ ]:
df_hurto1['fecha_hecho'] = pd.to_datetime(df_hurto1['fecha_hecho'])

In [ ]:
df_hurto1.insert(1, 'Año', df_hurto1['fecha_hecho'].dt.year)
df_hurto1.insert(2, 'Mes', df_hurto1['fecha_hecho'].dt.month)
df_hurto1.insert(3, 'Día', df_hurto1['fecha_hecho'].dt.day)
df_hurto1.insert(4, 'Hora', df_hurto1['fecha_hecho'].dt.hour.astype(str) + df_hurto1['fecha_hecho'].dt.minute.apply(lambda x: str(x).zfill(2)))

In [ ]:
df_hurto1.insert(4, 'Fin de semana',df_hurto1['fecha_hecho'].apply(lambda date: 'Sí' if date.weekday() >= 4 else 'No'))
# La ahora esta expresada en formato militar
df_hurto1['Hora'] = df_hurto1['Hora'].astype(int)

In [ ]:
df_hurto1['Año'] = df_hurto1['Año'].astype(object)
df_hurto1['Mes'] = df_hurto1['Mes'].replace({
                                              1:'Ene', 2:'Feb', 3:'Mar', 4:'Abr',
                                              5:'May', 6:'Jun', 7:'Jul', 8:'Ago',
                                              9:'Sep', 10:'Oct', 11:'Nov', 12:'Dic'
                                              })
df_hurto1['Mes'] = df_hurto1['Mes'].astype(object)
df_hurto1['Día'] = df_hurto1['Día'].astype(object)
df_hurto1['Fin de semana'] = df_hurto1['Fin de semana'].astype(object)
df_hurto1 = df_hurto1.drop(columns = 'fecha_hecho')

In [ ]:
df_hurto1.head()

#### **Outliers**

##### Metodo gráfico

In [ ]:
fig = sns.boxplot(data = df_hurto1, x = 'edad')
fig.set_xlabel('Edad')

##### Z-score

In [ ]:
# Calcular z-score para cada punto y obtener su valor absoluto
z_scores = zscore(df_hurto1['edad'])
abs_z_scores = np.abs(z_scores)

# Seleccionar los outliers usando un límite de 4.5

outliers_zscore1 = df_hurto1[abs_z_scores > 4.5]
outliers_zscore1['edad'].min()

Los métodos para identificar outliers no tiene en cuenta los datos que están a la izquierda del gráfico. Es por eso que decidimos revisarlos manualmente teniendo como criterio la edad mínima para obtener una licencia de conducir en Colombia que es de 16 años.

In [ ]:
df_fil1 = df_hurto1[df_hurto1['edad'] < 16]
len(df_fil1)/len(df_hurto1)*100

Estos datos atipicos corresponden al 2.35% del total de datos por lo cual decicimos eliminarlos.

In [ ]:
# Eliminación de filas cuya edad sea menor de 16
df_hurto1 = df_hurto1[df_hurto1['edad'] >= 16]

Se escoge eliminar los outliers a derecha del grafico mediante el método del Z-score con un valor de 4.5 como referencia. Es decir, los valores que contienen un Z-score mayor a 4.5 en la variable 'edad' serán eliminados.

In [ ]:
df_hurto1 = df_hurto1[abs_z_scores <= 4.5]

In [ ]:
# Reinicio de los indices
df_hurto1.reset_index(drop=True, inplace=True)

---
### **Base de datos de motos recuperadas**

In [ ]:
# Copia del DataFrame original para su manipulación
df_rec1 = df_rec

In [ ]:
df_rec1.shape

#### **Valores duplicados**

In [ ]:
df_rec1.duplicated().sum()

La base de datos no cuenta con valores duplicados.

#### **Valores nulos**

In [ ]:
df_rec1.isnull().sum()

La base de datos no cuenta con valores nulos.

#### **Redefinición y categorización de variables**

In [ ]:
nombre_com = {
    '1': 'POPULAR', '2': 'SANTA CRUZ', '3': 'MANRIQUE', '4': 'ARANJUEZ',
    '5': 'CASTILLA', '6': 'DOCE DE OCTUBRE', '7': 'ROBLEDO', '8': 'VILLA HERMOSA',
    '9': 'BUENOS AIRES', '10': 'LA CANDELARIA', '11': 'LAURELES', '12': 'LA AMERICA',
    '13': 'SAN JAVIER', '14': 'EL POBLADO', '15': 'GUAYABAL', '16': 'BELEN',
    '50': 'PALMITAS', '60': 'SAN CRISTOBAL', '70': 'ALTAVISTA', '80': 'SAN ANTONIO DE PRADO',
    '90': 'SANTA ELENA'
}

In [ ]:
unique_comunas = df_rec1['codigo_comuna'].unique()
results = []

for codigo in unique_comunas:
    matching_rows = df_rec1[df_rec1['codigo_comuna'] == codigo]
    sum_of_cantidad = matching_rows['cantidad'].sum()
    nombre_comuna = nombre_com.get(str(codigo))
    results.append([codigo, nombre_comuna, sum_of_cantidad])

# df_rec2 sera un DF donde esten almacenados todos los vehiculos recuperados a lo largo del tiempo segmentados por comuna
df_rec2 = pd.DataFrame(results, columns=['codigo_comuna', 'nombre_comuna', 'cantidad_recuperaciones'])
# Eliminación de los corregimos del DataFrame
corre = ['ALTAVISTA', 'SAN CRISTOBAL', 'SAN ANTONIO DE PRADO', 'SANTA ELENA', 'PALMITAS']
for i in corre:
  df_rec2.drop(df_rec2[df_rec2['nombre_comuna'] == i].index, inplace=True)

# Eliminación del valor 'SIN DATO'
df_rec2 = df_rec2[df_rec2['codigo_comuna'] != 'SIN DATO']

In [ ]:
df_rec2['nombre_comuna'].unique()

In [ ]:
df_rec2

In [ ]:
df_rec2['codigo_comuna'] = df_rec2['codigo_comuna'].replace({
                                              '1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8,
                                              '9':9, '10':10, '11':11, '12':12, '13':13, '14':14, '15':15,
                                              '16':16})
df_rec2['codigo_comuna'].astype(int)
df_rec2 = df_rec2.sort_values(by='codigo_comuna')
# Reinicio de los indices
df_rec2.reset_index(drop=True, inplace=True)
df_rec2

In [ ]:
# df_rec3 sera un DF donde esten almacenados todos los vehiculos recuperados año a año
df_rec3 = df_rec1.groupby('año')['cantidad'].sum().reset_index()
df_rec3.columns = ['año', 'total_cantidad']
df_rec3

### **Base de datos de Estratificación**

In [ ]:
# Copia del DataFrame original para su manipulación
df_estra_b = df_estratos_barrio
df_estra_b

####Información no relevante

La columna *'En Proceso de Estratificación'* contiene valores que no consideraremos para el estudio, por lo que decidimos eliminarla.

In [ ]:
df_estra_b = df_estra_b.drop(columns = ['En Proceso de Estratificación'])

#### Valores duplicados

In [ ]:
df_estra_b.duplicated().sum()

No encontramos valores duplicados en la base de datos.

#### Valores nulos

In [ ]:
df_estra_b.isnull().sum()

Los datos nulos que contiene la base datos dan cuenta de que en ese barrio no hay hogares pertenecientes a dicho estrato.

### Base de datos Estratificación modificada

Modificamos el nombre de las columnas '1.0', '2.0', '3.0', '4.0', '5.0', '6.0' y 'Predominante' para una mejor comprensión de estas.

In [ ]:
df_estra_b = df_estra_b.rename(columns={1.0: 'Estrato 1', 2.0: 'Estrato 2', 3.0: 'Estrato 3',
                                  4.0: 'Estrato 4', 5.0: 'Estrato 5', 6.0: 'Estrato 6',
                                  'Predominante': 'Estrato Predominante'})

df_estra_b

In [ ]:
# Eliminación de los corregimos del DataFrame
corre = ['ALTAVISTA', 'SAN CRISTOBAL', 'SAN ANTONIO DE PRADO', 'SANTA ELENA', 'PALMITAS']
for i in corre:
  df_estra_b.drop(df_estra_b[df_estra_b['Nombre Comuna'] == i].index, inplace=True)

In [ ]:
#Llenamos el dato faltante con un solo dato, en este caso el cero
df_estra_b.fillna(0)

Creamos un diccionario con las el nombre de los barrios por comuna para su manipulación

In [ ]:
df_estra_c = df_estratos_comuna
df_estra_c

In [ ]:
df_estra_c.info()

In [ ]:
#df_estra2a = df_estra2.drop(columns = ['Cantidad de predios	'])
df_estra_c = df_estra_c.drop('Cantidad de predios ', axis = 1)

In [ ]:
# Eliminación de los corregimos del DataFrame
corre = ['ALTAVISTA', 'SAN CRISTOBAL', 'SAN ANTONIO DE PRADO', 'SANTA ELENA', 'PALMITAS']
for i in corre:
  df_estra_c.drop(df_estra_c[df_estra_c['Nombre Comuna'] == i].index, inplace=True)

In [ ]:
df_estra_c

___
## Descarga de las bases de datos

In [ ]:
df_cam1.to_csv('Camaras.csv')
df_hurto1.to_csv('Hurtos.csv')
df_rec1.to_csv('Recuperacion1.csv')
df_rec2.to_csv('Recuperacion2.csv')
df_rec3.to_csv('Recuperacion3.csv')
df_estra_b.to_csv('Estratos barrios.csv')
df_estra_c.to_csv('Estratos comunas.csv')

In [ ]:
from google.colab import files
files.download('Camaras.csv')
files.download('Hurtos.csv')
# files.download('Recuperacion1.csv')
# files.download('Recuperacion2.csv')
# files.download('Recuperacion3.csv')
# files.download('Estratos barrios.csv')
# files.download('Estratos comunas.csv')

In [ ]:
df_hurto1.info()

## **CONSULTAS SQL**

* De la base de datos cámaras, acceder a la cantidad de cámaras de cada comuna.

* De la base de datos de hurtos seleccione los hurtos que se han registrado en cada comuna.

* Genere una tabla que dé cuenta de los hurtos que se han registrado en rangos de dos horas (Ej. de las 00:00 a las 02:00…).

* Genere una tabla que muestre la cantidad de recuperación, en el último año, de las comunas que cuentan con cámaras SIMM.

* Genere una tabla donde muestre la cantidad de hurtos entre semana y fin de semana teniendo en cuenta el rango horario de mayor y menor cantidad de hurtos.


---
# **Gráficos**

### Diagrama de torta

In [ ]:
conteo = df_hurto1['codigo_comuna'].value_counts()
conteo = dict(conteo)
conteo = dict(sorted(conteo.items()))
conteo

In [ ]:
explo = [0,0,0,0,0,0,0,0,0,0.2,0,0,0,0,0,0]

fig, ax = plt.subplots(figsize=(10, 10))
ax.pie(conteo.values(), labels=conteo.keys(), colors = ['#D3D3D3', 'aquamarine', '#1E90FF', 'teal'], explode = explo, autopct='%1.1f%%', startangle=180)
ax.axis('equal')    # De este modo la figura tendrá forma circular
fig.show()

___
### Linea de tiempo

#### Agrupación por mes hurtos totales

In [ ]:
df_hurto1

In [ ]:
grouped_df2 = df_hurto1.groupby(['Año', 'Mes']).size().reset_index(name='count')
grouped_df2['Mes'] = grouped_df2['Mes'].replace({
                                              'Ene':1, 'Feb':2, 'Mar':3, 'Abr':4,
                                              'May':5, 'Jun':6, 'Jul':7, 'Ago':8,
                                              'Sep':9, 'Oct':10, 'Nov':11, 'Dic':12
                                              })
grouped_df2['Fecha'] = grouped_df2.apply(lambda row: f"{row['Año']}-{row['Mes']}", axis=1)
grouped_df2['Fecha'] = pd.to_datetime(grouped_df2['Fecha'], format='%Y-%m')
print(grouped_df2)

In [ ]:
grouped_df2 = grouped_df2.sort_values(by='Fecha')
grouped_df2 = grouped_df2.drop(columns=['Año', 'Mes'])
grouped_df2 = grouped_df2.iloc[:, [1, 0]]
grouped_df2 = grouped_df2.reset_index(drop=True)
print(grouped_df2)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(grouped_df2['Fecha'], grouped_df2['count'], marker = 'o');
plt.title('Evolución de los hurtos en el tiempo', fontdict = {'fontsize':20, 'fontweight':'bold'})
plt.ylabel('Cantidad de hurtos', fontdict = {'fontsize':15})
plt.grid(axis = 'y', color = 'green', linestyle = 'dashed')
fig.show(warn=False)

#### Agrupación por mes hurtos por comuna

In [ ]:
df_hurto1['codigo_comuna'] = df_hurto1['codigo_comuna'].astype(int)

In [ ]:
grouped_dfv = df_hurto1.groupby(['Año', 'Mes', 'codigo_comuna']).size().reset_index(name='count')
grouped_dfv['Mes'] = grouped_dfv['Mes'].replace({
                                              'Ene':1, 'Feb':2, 'Mar':3, 'Abr':4,
                                              'May':5, 'Jun':6, 'Jul':7, 'Ago':8,
                                              'Sep':9, 'Oct':10, 'Nov':11, 'Dic':12
                                              })
grouped_dfv['Fecha'] = grouped_dfv.apply(lambda row: f"{row['Año']}-{row['Mes']}", axis=1)
grouped_dfv['Fecha'] = pd.to_datetime(grouped_dfv['Fecha'])
grouped_dfv = grouped_dfv.sort_values(by='Fecha')
grouped_dfv = grouped_dfv.drop(columns=['Año', 'Mes'])
grouped_dfv = grouped_dfv.reset_index(drop=True)
print(grouped_dfv)

In [ ]:
comunas = {}
for i in range(1, 17):
    comuna_name = 'comuna' + str(i)
    comunas[comuna_name] = grouped_dfv[grouped_dfv['codigo_comuna'] == i].reset_index(drop=True).sort_values(by='Fecha')

In [ ]:
# fig, ax = plt.subplots(figsize=(20, 6))
# for i in range(1,17):
#     ax.plot(comunas['comuna' + str(i)]['Fecha'], comunas['comuna' + str(i)]['count'],marker = 'o', label = 'Comuna ' + str(i));
# ax.legend(loc='upper left')
# plt.title('Evolución de los hurtos por comuna en el tiempo', fontdict = {'fontsize':15, 'fontweight':'bold'})
# fig.show(warn=False)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(comunas['comuna10']['Fecha'], comunas['comuna10']['count'],marker = 'o', label = 'Comuna 10');
ax.legend(loc='upper left')
plt.grid(axis = 'y', color = 'green', linestyle = 'dashed')
plt.title('Evolución de los hurtos por comuna en el tiempo', fontdict = {'fontsize':15, 'fontweight':'bold'})
fig.show(warn=False)

In [ ]:
conteo = df_hurto1['nombre_comuna'].value_counts()
conteo

In [ ]:
contcam =df_cam1['comuna'].value_counts()
contcam

---
### Diagrama de barras

#### Cantidad vs dia de la semana

In [ ]:
diagra = df_hurto1.groupby(['Año', 'Mes', 'Día']).size().reset_index(name='count')

diagra['Mes'] = diagra['Mes'].map({
    'Ene': 1, 'Feb': 2, 'Mar': 3, 'Abr': 4,
    'May': 5, 'Jun': 6, 'Jul': 7, 'Ago': 8,
    'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dic': 12
})

diagra['Fecha'] = pd.to_datetime(diagra.apply(lambda row: f"{row['Año']}-{row['Mes']}-{row['Día']}", axis=1))
diagra = diagra.drop(columns=['Año', 'Mes', 'Día']).reset_index(drop=True)
diagra = diagra.sort_values(by='Fecha')

diagra['Día'] = diagra['Fecha'].dt.day_name()
diagramadias = diagra.groupby('Día')['count'].sum().reset_index()

diagramadias['Día'] = diagramadias['Día'].map({
    'Monday': 'Lunes', 'Tuesday': 'Martes', 'Wednesday': 'Miércoles',
    'Thursday': 'Jueves', 'Friday': 'Viernes', 'Saturday': 'Sábado',
    'Sunday': 'Domingo'
})
diagramadias["Día"] = pd.Categorical(diagramadias["Día"], categories=[
    'Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo'
], ordered=True)
diagramadias = diagramadias.sort_values(by='Día')
diagramadias = diagramadias.reset_index(drop=True)

diagramadias


In [ ]:
fig = px.bar(diagramadias, x='Día', y='count')
fig.show()

#### Cantidad vs hombre-mujer

Diagrama de barras agrupadas que muestre según el genero la cantidad de hurtos que ocurren cada día de la semana, buscando una correlación temporal con los días de la semana para inducir el diagrama de hurtos por fin de semana


In [ ]:
diagrami = df_hurto1.groupby(['Año', 'Mes', 'Día', 'sexo']).size().reset_index(name='count')

diagrami['Mes'] = diagrami['Mes'].map({
    'Ene': 1, 'Feb': 2, 'Mar': 3, 'Abr': 4,
    'May': 5, 'Jun': 6, 'Jul': 7, 'Ago': 8,
    'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dic': 12
})

diagrami['Fecha'] = pd.to_datetime(diagrami.apply(lambda row: f"{row['Año']}-{row['Mes']}-{row['Día']}", axis=1))
diagrami = diagrami.drop(columns=['Año', 'Mes', 'Día']).reset_index(drop=True)
diagrami = diagrami.sort_values(by='Fecha')

diagrami['Día'] = diagrami['Fecha'].dt.day_name()
diagrami

In [ ]:
diagramasexo = diagrami.groupby(['Día', "sexo"])["count"].sum().unstack().reset_index()
diagramasexo = diagramasexo[['Día', 'Hombre', 'Mujer']]
diagramasexo['Día'] = diagramasexo['Día'].map({
    'Monday': 'Lunes', 'Tuesday': 'Martes', 'Wednesday': 'Miércoles',
    'Thursday': 'Jueves', 'Friday': 'Viernes', 'Saturday': 'Sábado',
    'Sunday': 'Domingo'
})
diagramasexo["Día"] = pd.Categorical(diagramasexo["Día"], categories=[
    'Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo'
], ordered=True)
diagramasexo = diagramasexo.sort_values(by='Día')
diagramasexo = diagramasexo.reset_index(drop=True)
diagramasexo

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(name='Mujer', x=diagramasexo['Día'], y=diagramasexo['Mujer']))
fig.add_trace(go.Bar(name='Hombre', x=diagramasexo['Día'], y=diagramasexo['Hombre']))

fig.show()

#### Cantidad por día de la semana según comuna

In [ ]:
diagrami = df_hurto1.groupby(['Año', 'Mes', 'Día', 'nombre_comuna']).size().reset_index(name='count')

diagrami['Mes'] = diagrami['Mes'].map({
    'Ene': 1, 'Feb': 2, 'Mar': 3, 'Abr': 4,
    'May': 5, 'Jun': 6, 'Jul': 7, 'Ago': 8,
    'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dic': 12
})

diagrami['Fecha'] = pd.to_datetime(diagrami.apply(lambda row: f"{row['Año']}-{row['Mes']}-{row['Día']}", axis=1))
diagrami = diagrami.drop(columns=['Año', 'Mes', 'Día']).reset_index(drop=True)
diagrami = diagrami.sort_values(by='Fecha')

diagrami['Día'] = diagrami['Fecha'].dt.day_name()
diagrami

In [ ]:
diagramacomuna = diagrami.groupby(['Día', 'nombre_comuna'])['count'].sum().unstack().reset_index()
diagramacomuna

#### Cantidad vs hora

In [ ]:
# Crear rangos de una hora
rangos= pd.cut(df_hurto1['Hora'], range(0, 2500, 100))
# Contar hurtos por rango de hora
hurtos_por_rango = rangos.value_counts()
# Ordenar hurtos por rango de hora
hurtos_por_rango = hurtos_por_rango.sort_index()
hurtos_por_rango

In [ ]:
s = pd.DataFrame(hurtos_por_rango)
s = s.reset_index().rename(columns={'index': 'indices', 'count': 'Hurtos'})

In [ ]:
s.plot(kind='bar', x='Hora', y='Hurtos', color= 'g')
plt.title('Hurtos por rango horario')
plt.xlabel('Rango horario')
plt.ylabel('Cantidad de hurtos')
plt.show()

#### Modalidad vs comuna

In [ ]:
df_hurto1['modalidad'].unique()

In [ ]:
barras = df_hurto1.groupby(['codigo_comuna', 'modalidad']).size().reset_index(name='count')
barras

### Grafico de radar

In [ ]:
from math import pi
# number of variable
categories = df_hurto1['codigo_comuna'].unique()
N = len(categories)

# We are going to plot the first line of the data frame.
# But we need to repeat the first value to close the circular graph:
values = df_hurto1['codigo_comuna'].value_counts().tolist()
values += values[:1]
values

# What will be the angle of each axis in the plot? (we divide the plot / number of variable)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

# Initialise the spider plot
ax = plt.subplot(111, polar=True)

# Draw one axe per variable + add labels
plt.xticks(angles[:-1], categories, color='r', size=5)

# Draw ylabels
ax.set_rlabel_position(0)
plt.yticks([2000, 4000, 6000, 8000, 10000, 12000], ["2K","4K","6K",'8K', '10K', '12K'], color='black', size=7)
plt.ylim(0,12600)

# Plot data
ax.plot(angles, values, linewidth=0.5, linestyle='solid', color= 'r')

# Fill area
ax.fill(angles, values, 'r', alpha=0.1)

# Show the graph
plt.show()

In [ ]:
import numpy as np
from scipy.optimize import fsolve

# Define the cubic equation
def cubic_equation(Q):
    return Q**3 - 10*Q**2 - 4000

# Use fsolve to find the roots
initial_guess = 10
Q_star = fsolve(cubic_equation, initial_guess)[0]

# Calculate the cost average (CMe) at Q_star
CMe_star = Q_star**2 - 20*Q_star + 100 + 8000/Q_star

# Calculate the price (P) at equilibrium
P_star = CMe_star

# Calculate the number of firms (n)
n_firms = (2500 - 3 * P_star) / Q_star

Q_star, P_star, n_firms